In [34]:
import pandas as pd

# Load your dataset
sales_data = pd.read_csv("Apparel_Sales_Transactions_Dataset_Weekly.csv")

# Drop 'Product_Name' column, as it is not needed for the forecasting model
sales_data_numeric = sales_data.drop(columns=['Product_Name'])


In [35]:
sales_data_numeric.head()

,W0,W1,W2,W3,W4,W5,W6,W7,W8,W9,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
0,42,48,38,43,35,39,36,38,49,46,...,46,41,34,39,44,46,46,41,57,73
1,47,42,24,55,42,23,41,51,45,29,...,29,25,38,46,46,34,34,45,41,26
2,35,34,40,36,45,37,43,47,34,39,...,52,36,43,31,37,38,42,28,28,26
3,38,32,36,37,42,33,56,37,39,31,...,38,30,36,37,34,39,44,31,25,16
4,28,43,40,43,54,33,39,30,38,38,...,43,39,34,33,41,43,35,30,29,27


In [36]:
sales_data_numeric.describe()

,W0,W1,W2,W3,W4,W5,W6,W7,W8,W9,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000,10.000000,10.000000,10.000000,10.000000
mean,38.500000,38.500000,40.500000,42.300000,41.600000,38.40000,39.800000,39.800000,39.800000,37.700000,...,40.700000,35.100000,35.200000,36.200000,38.700000,35.40000,37.200000,34.300000,32.300000,28.600000
std,8.540752,6.720615,8.746428,5.831905,6.995236,7.27553,9.126275,6.860515,4.491968,5.078276,...,6.165315,8.478076,5.349974,5.573748,8.219895,6.41526,5.050853,6.342975,10.392839,16.473548
min,26.000000,25.000000,24.000000,36.000000,31.000000,23.00000,30.000000,30.000000,34.000000,29.000000,...,29.000000,25.000000,23.000000,28.000000,29.000000,26.00000,31.000000,26.000000,23.000000,15.000000
25%,34.250000,34.500000,36.500000,37.750000,37.500000,34.00000,32.250000,37.000000,37.250000,35.750000,...,38.000000,27.000000,34.000000,32.250000,31.750000,30.00000,34.000000,30.000000,26.000000,20.500000
50%,37.500000,41.000000,40.000000,42.000000,41.000000,39.50000,38.000000,39.000000,38.000000,38.000000,...,41.000000,35.000000,35.500000,35.500000,38.500000,35.50000,35.500000,32.500000,28.500000,26.000000
75%,43.500000,42.750000,42.750000,44.500000,44.250000,43.50000,42.500000,44.000000,42.000000,39.750000,...,43.000000,40.500000,37.750000,39.750000,43.250000,38.75000,40.750000,39.500000,36.000000,27.750000
max,54.000000,48.000000,56.000000,55.000000,54.000000,47.00000,56.000000,51.000000,49.000000,46.000000,...,52.000000,50.000000,43.000000,46.000000,55.000000,46.00000,46.000000,45.000000,57.000000,73.000000


In [37]:
sales_data_numeric.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 52 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   W0      10 non-null     int64
 1   W1      10 non-null     int64
 2   W2      10 non-null     int64
 3   W3      10 non-null     int64
 4   W4      10 non-null     int64
 5   W5      10 non-null     int64
 6   W6      10 non-null     int64
 7   W7      10 non-null     int64
 8   W8      10 non-null     int64
 9   W9      10 non-null     int64
 10  W10     10 non-null     int64
 11  W11     10 non-null     int64
 12  W12     10 non-null     int64
 13  W13     10 non-null     int64
 14  W14     10 non-null     int64
 15  W15     10 non-null     int64
 16  W16     10 non-null     int64
 17  W17     10 non-null     int64
 18  W18     10 non-null     int64
 19  W19     10 non-null     int64
 20  W20     10 non-null     int64
 21  W21     10 non-null     int64
 22  W22     10 non-null     int64
 23  W23     10 non-nul

In [30]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Initialize a dictionary to store predictions for each product
extended_predictions = {}

# Loop through each product to forecast the next 10 weeks after week 51
for i in range(sales_data_numeric.shape[0]):
    # Get the sales data for each product
    product_sales = sales_data_numeric.iloc[i, :]

    # Fit a SARIMA model to the product's sales data
    sarima_model = SARIMAX(product_sales, 
                           order=(1, 1, 1),         # ARIMA parameters
                           seasonal_order=(1, 1, 1, 7),  # SARIMA seasonal parameters with weekly seasonality
                           enforce_stationarity=False, 
                           enforce_invertibility=False)
    
    # Train the SARIMA model
    sarima_result = sarima_model.fit(disp=False)

    # Forecast the next 10 weeks (weeks 52 to 61)
    forecast_extended = sarima_result.forecast(steps=10)
    
    # Store the forecasted values in the dictionary
    product_name = sales_data.iloc[i]['Product_Name']  # Get product name
    extended_predictions[product_name] = forecast_extended

# Convert predictions dictionary to a DataFrame for readability
extended_predictions_df = pd.DataFrame(extended_predictions)

# Display the forecast for the next 10 weeks for each product
print(extended_predictions_df)


/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:559: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  _index = to_datetime(index)
/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:559: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  _index = to_datetime(index)
/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/lib/python3.12/site-packages/statsmodels/tsa/base

    BreezeFit T- Shirts  Linen Shirts  CoolWave BoardShorts  Maxi Dresses  \
52            62.355610     38.788768             25.748608     24.021708   
53            58.666540     35.281111             28.356177     16.657055   
54            57.521772     30.643483             24.815913     15.498872   
55            57.378730     37.542282             31.528043     22.179747   
56            58.523149     36.242243             31.532551     17.804117   
57            61.843240     32.594090             26.573174     18.178227   
58            66.972234     33.304311             28.527635     15.092239   
59            64.767902     37.448328             24.113310     20.512775   
60            61.181885     34.968619             27.064279     14.826001   
61            60.038734     29.793256             23.496859     12.965948   

     Tank Top  PalmShade Shorts  Beach Towels  Wrap Skirts  Sleeveless Tops  \
52  30.757338         27.502534     32.606068    15.304815        22.3034

In [31]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Split data into training (weeks 0–41) and test (weeks 42–51) sets
train_data = sales_data_numeric.iloc[:, :42]  # First 42 weeks for training
test_data = sales_data_numeric.iloc[:, 42:52]  # Last 10 weeks for testing

# Initialize a dictionary to store the predictions and error metrics for each product
predictions = {}
mae_dict = {}
mse_dict = {}
rmse_dict = {}


# Loop through each product to train SARIMA and forecast sales
for i in range(train_data.shape[0]):
    product_sales = train_data.iloc[i]  # Sales data for the product (weeks 0–41)
    
    # Fit SARIMA model
    sarima_model = SARIMAX(product_sales, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7), enforce_stationarity=False, enforce_invertibility=False)
    sarima_result = sarima_model.fit(disp=False)
    
    # Forecast the next 10 weeks (weeks 42–51)
    forecast = sarima_result.forecast(steps=10)
    
    # Store predictions in the dictionary
    product_name = sales_data.iloc[i]['Product_Name']  # Get product name
    predictions[product_name] = forecast
    
    # Actual sales for the test period (weeks 42–51)
    actual_sales = test_data.iloc[i]
    
    # Calculate error metrics
    mae = mean_absolute_error(actual_sales, forecast)
    mse = mean_squared_error(actual_sales, forecast)
    rmse = np.sqrt(mse)
    
    
    # Store error metrics in dictionaries
    mae_dict[product_name] = mae
    mse_dict[product_name] = mse
    rmse_dict[product_name] = rmse
    

# Convert predictions dictionary to a DataFrame for readability
predictions_df = pd.DataFrame(predictions)

# Print out error metrics for each product
print("Mean Absolute Error (MAE) for each product:", mae_dict)
print("Mean Squared Error (MSE) for each product:", mse_dict)
print("Root Mean Squared Error (RMSE) for each product:", rmse_dict)


# Optionally, display the predictions
print("Predictions for weeks 42–51:", predictions_df)
print(test_data)
print(actual_sales)

/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:559: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  _index = to_datetime(index)
/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:559: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  _index = to_datetime(index)
/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/lib/python3.12/site-packages/statsmodels/tsa/base

Mean Absolute Error (MAE) for each product: {'BreezeFit T- Shirts': 10.33591132578398, 'Linen Shirts': 8.440677321306204, 'CoolWave BoardShorts': 9.261184965752125, 'Maxi Dresses': 8.041216201150373, 'Tank Top': 6.073791664824608, 'PalmShade Shorts': 7.707174663208977, 'Beach Towels': 6.056161761924367, 'Wrap Skirts': 11.274070801198539, 'Sleeveless Tops': 5.684137133553735, 'Sarongs': 7.511175415762418}
Mean Squared Error (MSE) for each product: {'BreezeFit T- Shirts': 164.4291223230495, 'Linen Shirts': 100.5231284919179, 'CoolWave BoardShorts': 125.72332396402126, 'Maxi Dresses': 97.95596834498409, 'Tank Top': 52.13612696210989, 'PalmShade Shorts': 87.58115876542033, 'Beach Towels': 80.04931409545142, 'Wrap Skirts': 181.65630103384015, 'Sleeveless Tops': 47.41472395707225, 'Sarongs': 76.4077046216525}
Root Mean Squared Error (RMSE) for each product: {'BreezeFit T- Shirts': 12.822991941159813, 'Linen Shirts': 10.026122305852741, 'CoolWave BoardShorts': 11.212641257260541, 'Maxi Dresse

In [33]:
Predict_trans = predictions_df.T
print(Predict_trans)
test_data.columns=test_data.columns.str.replace('W', '').astype(int)
print(test_data)
Predict = extended_predictions_df.T

import matplotlib.pyplot as plt


# Plot the actual vs predicted sales for a specific product
for i in range(0,9):
    product_index = i  # Replace with the index or name of the product you want to inspect
    plt.plot(test_data.iloc[product_index], label='Actual Sales')
    plt.plot(Predict_trans.iloc[product_index], label='Predicted test Sales')
    plt.plot(Predict.iloc[product_index], label='Predicted sales')
    plt.title(f'Sales Forecast vs Actual Sales ({predictions_df.columns[i]})')
    plt.xlabel('Week')
    plt.ylabel('Sales')
    plt.legend()
    plt.show()


                             42         43         44         45         46  \
BreezeFit T- Shirts   37.001536  39.253647  44.572471  42.571764  37.572981   
Linen Shirts          34.143417  29.239250  32.305833  32.132928  28.082137   
CoolWave BoardShorts  30.938704  26.750971  28.533598  24.694147  27.191786   
Maxi Dresses          30.636745  36.040939  32.142008  45.063370  33.553229   
Tank Top              38.460118  39.023700  35.724267  44.051106  30.347581   
PalmShade Shorts      35.002674  40.480663  37.177734  42.160582  37.449547   
Beach Towels          36.309207  33.730732  34.559619  33.645934  32.335049   
Wrap Skirts           43.826175  38.788542  46.426179  40.949777  40.379935   
Sleeveless Tops       33.291875  33.488242  29.775460  28.831633  33.852424   
Sarongs               25.732593  33.855784  28.015580  30.461316  33.210853   

                             47         48         49         50         51  
BreezeFit T- Shirts   35.907395  35.739459  36.40700

<class 'AttributeError'>: Can only use .str accessor with string values!